In [ ]:
# Install all necessary libraries
!pip3 install pillow pydicom tensorflow #python-varname

In [ ]:
# Import all necessary libraries

import csv
import numpy as np
import pandas as pd
import os
import seaborn as sns
import matplotlib.pyplot as plt
import torch
import torch.optim as optim
from torch import nn
import sys
import torch.nn.functional as F
from torch.utils.data import Dataset as TorchDataset
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models
from PIL import Image
import PIL.ImageOps
import math
import sys
import random
import collections
import requests
import glob
import pydicom as pdm
#import varname
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint
from tqdm.autonotebook import tqdm
from sklearn.utils import shuffle
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
import cv2
import gc
from tensorflow.python.keras.utils.data_utils import Sequence
from tensorflow.python.ops import array_ops

In [ ]:
np.seterr(divide='ignore', invalid='ignore')

## Define Constants

In [ ]:
# Input parameters
INPUT_DIR = "../input/rsna-intracranial-hemorrhage-detection/rsna-intracranial-hemorrhage-detection/"


# Preprocessing Parameters
HU_RESCALE = True
WINDOW = True
NORMALIZE = True

In [ ]:
# Model Structure Parameters
BATCH_SIZE = 16  # can increase with greater gpu strength
VAL_BATCH_RATIO = .2  # ratio of (training entries):(validation entries)
criterion = nn.CrossEntropyLoss()
#optimizer = torch.optim.Adam()
#Activation Function = F.relu(): relu usually better than sigmoid, esp for CNNs
INITIAL_LR = 4e-5  # initial learning rate may decay over time - this is max LR
MODEL_PATH = '/kaggle/input/baseline-resnext50/resnext50_10.pth'

PYTORCH_EPOCHS = 2
KERAS_EPOCHS = 10


# Model Analysis Parameters
IMAGE_MODE = '2d'
MANUAL_ANY = False
HEMORRHAGE_TYPES = ('epidural', 'intraparenchymal', 'intraventricular', 'subarachnoid', 'subdural')

N_DATA = 5000  # full data = 752803 entries
VALID_SIZE = math.ceil(N_DATA*VAL_BATCH_RATIO) #validation data size
N_THREADS = 4  # num parallell ops
SHUFFLE = True

SUBMISSION_SIZE = 100

KERAS = True

#Keras Image Ranges
KERAS_VALID_IMAGE_START = 1000
KERAS_VALID_IMAGE_END = 1200
KERAS_TRAIN_IMAGE_START = 0
KERAS_TRAIN_IMAGE_END = 1000

In [ ]:
# Layer Parameters
IN_CHANNELS = 1  # B&W = 1, Greyscale = 2, RGB = 3

CONV1_CHANNELS = 512  # depth of conv layer 1 – one channel per pixel
CONV1_KERN = 5  # kernal length and width
CONV1_STRIDE = 1  # kernal must be able to move this much and cover all channels
CONV1_PAD = 0  # padding for conv1

CONV2_CHANNELS = 50
CONV2_KERN = 3
CONV2_STRIDE = 1
CONV2_PAD = 0

FC1_NEURONS = 500

POOL1_X = 2
POOL1_Y = 2

POOL2_X = 2
POOL2_Y = 2

DR1_PROB = .2

In [ ]:
# Set device for training the model (If gpu available - GPU, otherwise - CPU)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

## Pandas Helper Class

In [ ]:
class PdFuncs():
    
    @staticmethod
    def row_at_loc(df, val, search_col):
        return df.loc[df[search_col] == val]

    @staticmethod
    def extract_row_value(row, idx):
        try:
            return row.values.flatten().tolist()[idx]
        except AttributeError:
            return row.flatten().tolist()[idx]

    @staticmethod
    def extract_col_value():
        pass

    @staticmethod
    def extract_value(df, row, col):
        try:
            return df.iat(row, col)
        except TypeError:
            return df.at(row, df.columns[col])


    @staticmethod
    def match_extract(df, val, search_col, extract_col):
        row = PdFuncs.row_at_loc(df, val, search_col)
        try:
            return PdFuncs.extract_row_value(row, extract_col)
        except TypeError:
            return PdFuncs.extract_row_value(row, df.columns.get_loc(extract_col))

## Formula Calculations

In [ ]:
def output_size(inp, kern, stride, pad):  # verify dimensions, this is 3d (ret, ret, filters)
    return ((inp - kern + 2*pad)//stride) + 1

In [ ]:
def calc_any(): pass

## Processing

In [ ]:
# List and define paths constants

TRAIN = INPUT_DIR + "stage_2_train/"
TRAIN_CSV = INPUT_DIR + "stage_2_train.csv"
TRAIN_EDIT_CSV = "/kaggle/working/test.csv"
TEST = INPUT_DIR + "stage_2_test/"
TEST_CSV = INPUT_DIR + "stage_2_sample_submission.csv"
TEST_DIR = INPUT_DIR
os.listdir(INPUT_DIR)

In [ ]:
# Delete corrupted image IDs

corrupted_images = ('ID_6431af929_epidural', 'ID_6431af929_intraparenchymal', 'ID_6431af929_intraventricular', 'ID_6431af929_subarachnoid', 'ID_6431af929_subdural', 'ID_6431af929_any')

with open(TRAIN_CSV, 'r') as inp, open(TRAIN_EDIT_CSV, 'w') as out:
    writer = csv.writer(out)
    for row in csv.reader(inp):
        if row[0] not in corrupted_images:
            writer.writerow(row)

In [ ]:
def preprocess(dcm):
    m = dcm.RescaleSlope
    b = dcm.RescaleIntercept
    c = dcm.WindowCenter
    w = dcm.WindowWidth
    
    dcm = dcm.pixel_array
    
    # Convert to HU Units
    if HU_RESCALE:
        dcm = m*dcm + b
    
    # Window DCM
    if WINDOW:
        if isinstance(c, pdm.multival.MultiValue):
            c = c[0]
        if isinstance(w, pdm.multival.MultiValue):
            w = w[0]
    bounds = (c - w//2, c + w//2)
    dcm = np.clip(dcm, bounds[0], bounds[1])
    dcm = np.expand_dims(dcm, axis=2)
#     dcm.resize((512, 512, 2))
#     print(dcm.shape)
    # Normalize Pixel Array
    if NORMALIZE:
        dcm = 255*(dcm - dcm.min()) / (dcm.max() - dcm.min())
        
    return dcm

In [ ]:
def read_dcm(image_id, directory=TRAIN, mode="image"):
    dcm = pdm.dcmread(directory+f'ID_{image_id}.dcm')
  
    if mode == "image":
        return preprocess(dcm)
    if mode == "patient":
        return dcm.PatientID
    if mode == "angle":
        return dcm.ImagePositionPatient  #https://stackoverflow.com/questions/30814720/dicom-and-the-image-position-patient
    if mode == "orientation":
        return dcm.ImageOrientationPatient

In [ ]:
def display_dcm(arr):
    arr = np.squeeze(arr, axis=(2, ))
    #print(arr.shape)
    #image = Image.fromarray(np.asarray(arr))
    plt.axis('off')
    plt.imshow(arr, cmap=plt.cm.bone)

In [ ]:
dcm1 = read_dcm('8d3864098')
display_dcm(dcm1)
# print(dcm1)

In [ ]:
dcm2 = read_dcm('d3e935321')
display_dcm(dcm2)

In [ ]:
dcm1_pat = read_dcm('8d3864098', mode="patient")
dcm1_pat

## Read & Display Data

In [ ]:
class CSV_Sampler():
    def __init__(self, path, n=BATCH_SIZE, mode=IMAGE_MODE, incl_label=True):
        self.mode = mode
        self.unclassified_fnames = pd.DataFrame()
        self.classified_fnames = pd.DataFrame()
        
        self.IMGpath = ""
        if path == TRAIN_CSV: 
            self.IMGpath = TRAIN
        elif path == TEST_CSV:
            self.IMGpath = TEST

        self.incl_label=True
        self.unclassified_fnames = self.pull_data(path)
        self.classified_fnames = self.categorize_data(self.unclassified_fnames)
        
        self.IDs = self.sample_ids(self.classified_fnames, n, mode)
        self.IMGs = self.sample_images(self.IDs, mode)

    def __call__(self):
        pass

    def __getitem__(self, key):
        return torch.from_numpy(self.IMGs[key])

    def __len__(self):
        return len(self.IMGs)

    def __str__(self):
        return str(self.IMGs)
    
    @staticmethod
    def classify_image_category(x):
        return "_".join(x.split("_", 2)[2:])

    @staticmethod
    def retrieve_image_id(x):
        return "_".join(x.split("_", 2)[1:2])

    @staticmethod
    def retrieve_patient_id(x):
        return read_dcm(x, directory=self.IMGpath, mode="patient")


    def pull_data(self, path):
        df = pd.read_csv(path)
        df.rename(columns={'Label':'Category'}, inplace=True)
        if self.IMGpath == TRAIN:
            df['Category'] = np.where(df.Category == 1, df['ID'].apply(self.classify_image_category), 'X')
        elif self.IMGpath == TEST:
            df['Category'] = np.where(df.Category == 0.5, df['ID'].apply(self.classify_image_category), "")
        df['ID'] = df['ID'].apply(self.retrieve_image_id)
        # df.insert(2, "Patient", df['ID'].apply(cls.retrieve_patient_id))  # applies to all here but causes performance issues

        return df

    def categorize_data(self, data):
        df = self.unclassified_fnames
        fnames = pd.crosstab(df['ID'], df['Category']).astype(int).rename_axis(index=None,columns=None)
        fnames.reset_index(inplace=True)
        fnames.rename(columns={'index':'ID'}, inplace=True)
        if self.IMGpath == TRAIN:
            del fnames['X']
        elif self.IMGpath == TEST:
            fnames.drop_duplicates('ID')
            fnames = fnames.replace(True, 'Unknown')  # broken

        return fnames

    def label_image(self, id):
        df = self.classified_fnames
        label = []

        if MANUAL_ANY:
            pass
        else:
            for i in range(6):
                label.append(int(PdFuncs.match_extract(df, id, 'ID', i+1)))
      

        # any = df.loc[df['ID'] == id][1]
        # epidural = df.loc[df['ID'] == id][2]
        # intraparenchymal = df.loc[df['ID'] == id][3]
        # intraventricular = df.loc[df['ID'] == id][4]
        # subarachnoid = df.loc[df['ID'] == id,'subarachnoid']
        # subdural = df.loc[df['ID'] == id,'subdural']

        # # data = {
        # #     'any': any,
        # #     'epidural': epidural,
        # #     'intraparenchymal': intraparenchymal,
        # #     'intraventricular': intraventricular,
        # #     'subarachnoid': subarachnoid,
        # #     'subdural': subdural,
        # # }

        # return [any, epidural, intraparenchymal, 
        #         intraventricular, subarachnoid, subdural]

        return label
    

    def sample_ids(self, df, n, mode):
        if mode == '2d':  # 1 image per patient
            ids = set()
            pats = set()

        for _ in range(n):
            while True:
                row = df.sample()
                imgID = PdFuncs.extract_row_value(row, 0)
                if imgID in ids:
                    continue
                elif read_dcm(imgID, directory=self.IMGpath, mode="patient") in pats:
                    continue
                else:
                    ids.add(imgID)
                    pats.add(read_dcm(imgID, directory=self.IMGpath, mode="patient"))
                    break

            return ids

        if mode == '3d': 
            pass

    def sample_images(self, ids, mode):
        lab_imgs = []

        if mode == '2d':
            for id in ids:
                if self.incl_label:
                    img = read_dcm(id, directory=self.IMGpath)
                    lab = self.label_image(id)
                    lab_imgs.append([img, lab])
#                 else:
           

            return lab_imgs

        if mode == '3d':
            pass

In [ ]:
training_data = CSV_Sampler(TRAIN_CSV)
validation_data = CSV_Sampler(TRAIN_CSV, n=math.ceil(VAL_BATCH_RATIO*BATCH_SIZE))

In [ ]:
training_data.classified_fnames

In [ ]:
training_data.classified_fnames.shape

In [ ]:
# Multilabel logarithmic loss function

class FocalLoss(nn.Module):
    def __init__(self, alpha=1, gamma=2, logits=False, reduce=True):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.logits = logits
        self.reduce = reduce

    def forward(self, inputs, targets):
        if self.logits:
            BCE_loss = F.binary_cross_entropy_with_logits(inputs, targets, reduce=False)
        else:
            BCE_loss = F.binary_cross_entropy(inputs, targets, reduce=False)
        pt = torch.exp(-BCE_loss)
        F_loss = self.alpha * (1-pt)**self.gamma * BCE_loss

        if self.reduce:
            return torch.mean(F_loss)
        else:
            return F_loss

# EDA (Exploratory Data Analysis)

### Label Frequency

In [ ]:
df_train = pd.read_csv(TRAIN_CSV)
sns.countplot(df_train.Label).set_title("Label Frequency")

### Hemmorhage Frequency

In [ ]:
df_train['Sub_type'] = df_train['ID'].str.split("_", n = 3, expand = True)[2]
df_train['PatientID'] = df_train['ID'].str.split("_", n = 3, expand = True)[1]
#df_train.head()

In [ ]:
gbSub = df_train.groupby('Sub_type').sum()
gbSub

In [ ]:
freq = sns.barplot(y=gbSub.index, x=gbSub.Label, palette="deep")
freq.set_title('Hemmorhage Frequency')
freq.set_xlabel('Frequency')
freq.set_ylabel('Subtype')
freq

### Subtype Frequency

In [ ]:
fig=plt.figure(figsize=(10, 8))

frq = sns.countplot(x="Sub_type", hue="Label", data=df_train)
frq.set_xlabel('Subtype')
frq.set_ylabel('Frequency')

plt.title("Subtype Frequency")

frq

In [ ]:
def window_image(img, window_center,window_width, intercept, slope, rescale=True):

    img = (img*slope +intercept)
    img_min = window_center - window_width//2
    img_max = window_center + window_width//2
    img[img<img_min] = img_min
    img[img>img_max] = img_max
    
    if rescale:
        # Extra rescaling to 0-1, not in the original notebook
        img = (img - img_min) / (img_max - img_min)
    
    return img
    
def first_in_field(x):
    #get x[0] as in int is x is a 'pydicom.multival.MultiValue', otherwise get int(x)
    if type(x) == pdm.multival.MultiValue:
        return int(x[0])
    else:
        return int(x)

def get_windowing(data):
    dicom_fields = [data[('0028','1050')].value, #window center
                    data[('0028','1051')].value, #window width
                    data[('0028','1052')].value, #intercept
                    data[('0028','1053')].value] #slope
    return [first_in_field(x) for x in dicom_fields]

In [ ]:
def view_images(images, title = '', aug = None, path=TRAIN):
    width = 5
    height = 2
    fig, axs = plt.subplots(height, width, figsize=(15,5))
    
    for im in range(0, height * width):
        ''''
        i = im // width
        j = im % width
        axs[i,j].imshow(image, cmap=plt.cm.bone) 
        axs[i,j].axis('off')'''''
        
        data = pdm.read_file(os.path.join(path,'ID_'+images[im]+ '.dcm'))
        image = data.pixel_array
        window_center , window_width, intercept, slope = get_windowing(data)
        image_windowed = window_image(image, window_center, window_width, intercept, slope)


        i = im // width
        j = im % width
        axs[i,j].imshow(image_windowed, cmap=plt.cm.bone) 
        axs[i,j].axis('off')
        
        
    plt.suptitle(title)
    plt.show()

In [ ]:
view_images(df_train[(df_train['Sub_type'] == 'epidural') & (df_train['Label'] == 1)][:10].PatientID.values, title = 'Epidural Hemmorhages')

In [ ]:
view_images(df_train[(df_train['Sub_type'] == 'intraparenchymal') & (df_train['Label'] == 1)][:10].PatientID.values, title = 'Intraparenchymal Hemmorhages')

In [ ]:
view_images(df_train[(df_train['Sub_type'] == 'intraventricular') & (df_train['Label'] == 1)][:10].PatientID.values, title = 'Intraventricular Hemmorhages')

In [ ]:
view_images(df_train[(df_train['Sub_type'] == 'subarachnoid') & (df_train['Label'] == 1)][:10].PatientID.values, title = 'Subarachnoid Hemmorhages')

In [ ]:
view_images(df_train[(df_train['Sub_type'] == 'subdural') & (df_train['Label'] == 1)][:10].PatientID.values, title = 'Subdural Hemmorhages')

In [ ]:
view_images(df_train[(df_train['Sub_type'] == 'any') & (df_train['Label'] == 1)][:10].PatientID.values, title = 'Hemmorhages of Any Type')

# Pytorch implementation

## Data Preparation

In [ ]:
# Train and validation dataframe preparation 

df_train = pd.read_csv(TRAIN_CSV)
# df_train = pd.read_csv(TRAIN_EDIT_CSV) # Use to train the whole data
df_train[['id', 'img', 'subtype']] = df_train['ID'].str.split('_', n=3, expand=True)
df_train['img'] = 'ID_' + df_train['img'] 

df_train.drop_duplicates(inplace=True)
df_train = df_train.pivot(index='img', columns='subtype', values='Label').reset_index()
df_train['path'] = TRAIN + df_train['img'] + '.dcm'
# print(len(df_train))
df_valid = df_train
df_train  = df_train.iloc[:N_DATA]  # train first N_DATA images
df_valid = df_valid.iloc[N_DATA:N_DATA + VALID_SIZE] # initialize the validation dataframe
df_valid.reset_index(inplace=True)
del df_valid['index']
#df_train.rename(columns={'subtype':'idx'}, inplace=True)
df_train.head()
# len(df_train)

In [ ]:
# Test dataframe preparation

df_test = pd.read_csv(TEST_CSV)
df_test[['id','img','subtype']] = df_test['ID'].str.split('_', expand=True)
df_test['img'] = 'ID_' + df_test['img']
df_test = df_test[['img', 'Label']]
df_test['path'] = TEST + df_test['img'] + '.dcm'
df_test.drop_duplicates(inplace=True)

df_test = df_test.reset_index(drop=True)
df_test.head()

## Dataset class

In [ ]:
class Dataset(TorchDataset):
    """Dataset preparation class. Contains a list of images in appropriate format"""
    def __init__(self, df, labels):
        self.data = df
        self.labels = labels

    def __len__(self):
        """The size of dataset"""
        return len(self.data)

    def __getitem__(self, index):
        """Get element by its index as a numpy array"""
        
        img_name = self.data.loc[index, 'path']   
        
        img_dcm = pdm.read_file(img_name)
        img = Dataset.brain_window(img_dcm)
        img = cv2.resize(img, (200,200))
        
        img = np.stack((img,)*3, axis=-1)  # add another dimension
        img = np.transpose(img, (2, 1, 0))  # fix dimension sizes
    
                
        if self.labels:        
            labels = torch.tensor(
                self.data.loc[index, ['epidural', 'intraparenchymal', 'intraventricular', 'subarachnoid', 'subdural', 'any']])
            return {'image': img, 'labels': labels}   
        else:
            return {'image': img}
  
    @staticmethod      
    def brain_window(img):
        """Process .dcm formatted images. All cells should are windowed between 0 and 80, then the image is normalized"""
        window_min = 0
        window_max = 80
        _, _, intercept, slope = Dataset.get_windowing(img)
        img = img.pixel_array.astype('float32')
        img = img * slope + intercept
        img = np.clip(img, window_min, window_max)  # Window
        img = (img - np.min(img)) / 1e-5 + (np.max(img) - np.min(img))  # Normalize, 1e-5 so now div by 0
        return img
    
    @staticmethod
    def get_windowing(data):
        """Get data from .dcm image"""
        dicom_fields = [data[('0028','1050')].value, #window center
                        data[('0028','1051')].value, #window width
                        data[('0028','1052')].value, #intercept
                        data[('0028','1053')].value] #slope
        return [Dataset.first_in_field(x) for x in dicom_fields]
  
    @staticmethod
    def first_in_field(x):
        if type(x) == pdm.multival.MultiValue:
            return int(x[0])
        else:
            return int(x)

In [ ]:
# Set parameters for training

params = {'batch_size': BATCH_SIZE,
          'shuffle': SHUFFLE,
          'num_workers': N_THREADS}

# Create Dataset based on our dataframe

train_dataset = Dataset(df= df_train, labels=True)
test_dataset = Dataset(df= df_test, labels=False)
valid_dataset = Dataset(df = df_valid, labels=True)

# Create DataLoader based on our dataset

data_train_generator = DataLoader(train_dataset, **params)
data_test_generator = DataLoader(test_dataset,**params)
data_valid_generator = DataLoader(valid_dataset, **params)

## Example Images

In [ ]:
def display_images(dataloader, num_images = 5):
    batch = next(iter(dataloader))
    fig, axs = plt.subplots(1, num_images, figsize=(15, 5))
    
    for i in np.arange(num_images):
        axs[i].axis("off")
        axs[i].imshow(batch['image'][i][0].numpy(), cmap=plt.cm.bone)

### Training Images

In [ ]:
display_images(data_train_generator)

### Test Images

In [ ]:
display_images(data_test_generator)

## Create Model

In [ ]:
# Set resnet pretrained model, add a linear layer as the last layer because we have 6 classes, not 1000

model0 = models.resnext50_32x4d(pretrained=True)  # 1d batch size x 3d images
model = torch.nn.Sequential(model0, torch.nn.Linear(1000, 6) )  # add linear layer for output of 6, 1000 is default resnet output

# Set device (GPU or CPU) and loss function

model = model.to(device)
criterion = torch.nn.BCEWithLogitsLoss()  # logarithmic loss BCE

In [ ]:
def model_summary_pytorch(model):
    layers = [module for module in model.modules() if type(module) != nn.Sequential]
    print(layers)

In [ ]:
model_summary_pytorch(model)

In [ ]:
# Set number of epochs and define optimizer

n_epochs = PYTORCH_EPOCHS
optimizer = optim.Adam(model.parameters(), lr=INITIAL_LR)  # maybe implement changing LR
epochs = list(range(1, n_epochs+1))

try:
    model.load_state_dict(torch.load(MODEL_PATH))
    torch.save(model.state_dict(), 'resnext50_0.pth') 
except Exception:  # figure out exception
    print('Resnet loaded')

## Train Model

### Performance Analysis

In [ ]:
def show_model_report(actual, predicted, mode):
    print(mode + ' mode report :\n')
    actual = actual.cpu().numpy()
    predicted = predicted.cpu().detach().numpy()
    #print(actual)
    #print(predicted)
#     actual = np.add(np.argmax(actual, axis = 1), 1).tolist()
    actual = np.argmax(actual, axis = 1).tolist()
    predicted = np.argmax(predicted, axis = 1).tolist()
    matrix = confusion_matrix(actual, predicted)
    #report = classification_report(actual, predicted)
    print('Confusion Matrix :')
    print(matrix)
    #print('Report :')
    #print(report)

### Training

In [ ]:
# Standard pytorch training procedure

training_losses = []
val_losses = []
for epoch in range(1, n_epochs+1):
    
    print(f'Epoch {epoch}/{n_epochs}')
    print('-' * 10)

    model.train()    
    tr_loss = 0
    
    tk_train = tqdm(data_train_generator, desc='Images Processed')  # data enumerator
    
    for step, batch in enumerate(tk_train):
        
        inputs = batch["image"]
        labels = batch["labels"]

        inputs = inputs.to(device, dtype=torch.float)
        labels = labels.to(device, dtype=torch.float)

        outputs = model(inputs)
        loss = criterion(outputs, labels)
                
        loss.backward()

        optimizer.step()
        optimizer.zero_grad()
     
        tr_loss += loss.item()
        
        # Save scores for confusion matrix
            
#     torch.save(model.state_dict(), f'resnext50_{epoch}.pth') 
    if(epoch == n_epochs):
        show_model_report(labels, outputs, mode='training')
    
    model.eval()
    val_loss = 0
    tk_valid = tqdm(data_valid_generator, desc='Images Processed')
    
    for step, batch in enumerate(tk_valid):
        inputs = batch["image"]
        labels = batch["labels"]

        inputs = inputs.to(device, dtype=torch.float)
        labels = labels.to(device, dtype=torch.float)

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        
        optimizer.zero_grad()
        
        val_loss += loss.item()
    
    if(epoch == n_epochs):
        show_model_report(labels, outputs, mode='validation')
    
    epoch_train_loss = tr_loss / len(data_train_generator)
    epoch_val_loss = val_loss / len(data_valid_generator)
    training_losses.append(epoch_train_loss)
    val_losses.append(epoch_val_loss)
    print('Training Loss: {:.7f}'.format(epoch_train_loss))
    print('Validation Loss: {:.7f}'.format(epoch_val_loss))

In [ ]:
fig = sns.lineplot(epochs, training_losses)
fig.set_xlabel('Epoch')
fig.set_ylabel('Training Loss')
fig.set_xticks(list(range(1, PYTORCH_EPOCHS+1)))
fig

In [ ]:
fig = sns.lineplot(epochs, val_losses)
fig.set_xlabel('Epoch')
fig.set_ylabel('Validation Loss')
fig.set_xticks(list(range(1, PYTORCH_EPOCHS+1)))
fig

In [ ]:
torch.save(model.state_dict(), f'resnext50_improved.pth')

### Predict a random image

In [ ]:
def predict_image(image_id):
    df = df_test.loc[df_test['img'] == 'ID_'+image_id]
    print(df)
    dataset = Dataset(df = df, labels=False)
    dataloader = DataLoader(dataset, **params)
    for param in model.parameters():
        param.requires_grad = False
        model.eval()

    test_pred = np.zeros((len(dataset) * 6, 1))

    tk_test = tqdm(dataloader)

    for i, batch in enumerate(tk_test):
        batch = batch["image"]
        print(batch.shape)
        batch = batch.to(device, dtype=torch.float)
        with torch.no_grad():
            pred = model(batch)
            test_pred[(i * 1 * 6):((i + 1) * 1 * 6)] = torch.sigmoid(pred).detach().cpu().reshape((len(batch) * 6, 1))
    dictionary = {'ID': [image_id + '_epidural', image_id + '_intraparenchymal', image_id + '_intraventricular', image_id + '_subarachnoid', image_id + '_subdural', image_id + '_any'], 'Label': [0, 0, 0, 0, 0, 0]}
    submission = pd.DataFrame(dictionary) 
    submis = pd.concat([submission.drop(columns=['Label']), pd.DataFrame(test_pred)], axis=1)
    submis.columns = ['ID', 'Label']
    #print(submis)
    return submis

In [ ]:
predict_image('0fbf6a978')

### Predict Test Set

In [ ]:
for param in model.parameters():
    param.requires_grad = False
    
model.eval()

test_pred = np.zeros((len(test_dataset) * 6, 1))

tk_test = tqdm(data_test_generator)

for i, batch in enumerate(tk_test):
    if i == SUBMISSION_SIZE//BATCH_SIZE:
        break
    
    batch = batch["image"]
    batch = batch.to(device, dtype=torch.float)
    #print(batch.shape)

    with torch.no_grad():
        
        pred = model(batch)
        #print(pred.shape)
        test_pred[(i * BATCH_SIZE * 6):((i + 1) * BATCH_SIZE * 6)] = torch.sigmoid(
            pred).detach().cpu().reshape((len(batch) * 6, 1))
        

In [ ]:
submission =  pd.read_csv(TEST_CSV)
submission = pd.concat([submission.drop(columns=['Label']), pd.DataFrame(test_pred)], axis=1)
submission.columns = ['ID', 'Label']

submission.to_csv('submission.csv', index=False)

In [ ]:
SUBMIT_CSV = '/kaggle/working/submission.csv'
df_submit = pd.read_csv(SUBMIT_CSV)
df_submit[:50]

# Keras implementation

In [ ]:
if not KERAS: raise Exception('Keras implementation not selected')

In [ ]:
train = pd.read_csv(TRAIN_CSV)
train.shape
read_data = train.copy()
train.head()

In [ ]:
train['filename'] = train['ID'].apply(lambda x: "ID_" + x.split('_')[1] + ".dcm")
train['type'] = train['ID'].apply(lambda x: x.split('_')[2])
train.head()


In [ ]:
train = train[['Label', 'filename', 'type']].drop_duplicates().pivot(
    index='filename', columns='type', values='Label').reset_index()
train.head()

In [ ]:
train = shuffle(train)
train_sample=train
train_sample2=train_sample.reset_index(drop=True)

In [ ]:
train_sample2.head()

In [ ]:
len(train_sample2)

In [ ]:
yvals=pd.DataFrame(train_sample2,columns=['any','epidural','intraparenchymal','intraventricular','subarachnoid','subdural'])
yvals.head()

In [ ]:
xhead=pd.DataFrame(train_sample2,columns=['filename'])
xhead.head()

In [ ]:
def first_in_field(x):
    #get x[0] as in int is x is a 'pydicom.multival.MultiValue', otherwise get int(x)
    if type(x) == pdm.multival.MultiValue:
        return int(x[0])
    else:
        return int(x)

In [ ]:
def get_windowing(data):
    dicom_fields = [data[('0028','1050')].value, #window center
                    data[('0028','1051')].value, #window width
                    data[('0028','1052')].value, #intercept
                    data[('0028','1053')].value] #slope
    return [first_in_field(x) for x in dicom_fields]

In [ ]:
class DataGenerator(Sequence):
    'Generates data for Keras'
    def __init__(self, list_IDs_labels, batch_size=100, dim=(512,512)):
        'Initialization'
        self.dim = dim
        self.batch_size = batch_size
        self.list_IDs = list_IDs_labels
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indices of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_temp = [self.list_IDs['filename'][k] for k in indexes]
        list_label_temp=[[int(self.list_IDs['any'][i]),int(self.list_IDs['epidural'][i]),int(self.list_IDs['intraparenchymal'][i]),int(self.list_IDs['intraventricular'][i]),int(self.list_IDs['subarachnoid'][i]),int(self.list_IDs['subdural'][i])] for i in indexes]
        # Generate data
        X, y = self.__data_generation(list_IDs_temp,list_label_temp)
        X = tf.cast(X, tf.float32)
        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))

    def __data_generation(self, list_IDs_temp,list_label_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        X = []
        y = []

        # Generate data
        for i, ID in enumerate(list_IDs_temp):
            # Store sample
            ds=pdm.dcmread(TRAIN +list_IDs_temp[i] )
            temp=ds.pixel_array
            window_center , window_width, intercept, slope = get_windowing(ds)
            img = window_image(temp, 50, 100, intercept, slope)
            resized = cv2.resize(img, (200, 200))
            X.append(resized)       
        X=np.array(X).reshape(-1,200,200,1)
        y_train=np.asarray(list_label_temp) 
        return X,y_train


In [ ]:
valid=train_sample2[KERAS_VALID_IMAGE_START:KERAS_VALID_IMAGE_END]
valid=valid.reset_index(drop=True)
valid.head()

In [ ]:
traingen=DataGenerator(train_sample2[KERAS_TRAIN_IMAGE_START:KERAS_TRAIN_IMAGE_END])
validgen=DataGenerator(valid)

In [ ]:
def focal_loss(prediction_tensor, target_tensor, weights=None, alpha=0.25, gamma=2):
    r"""Compute focal loss for predictions.
        Multi-labels Focal loss formula:
            FL = -alpha * (z-p)^gamma * log(p) -(1-alpha) * p^gamma * log(1-p)
                 ,which alpha = 0.25, gamma = 2, p = sigmoid(x), z = target_tensor.
    Args:
     prediction_tensor: A float tensor of shape [batch_size, num_anchors,
        num_classes] representing the predicted logits for each class
     target_tensor: A float tensor of shape [batch_size, num_anchors,
        num_classes] representing one-hot encoded classification targets
     weights: A float tensor of shape [batch_size, num_anchors]
     alpha: A scalar tensor for focal loss alpha hyper-parameter
     gamma: A scalar tensor for focal loss gamma hyper-parameter
    Returns:
        loss: A (scalar) tensor representing the value of the loss function
    """
    prediction_tensor = tf.dtypes.cast(prediction_tensor, dtype=tf.float32)
    sigmoid_p = tf.nn.sigmoid(prediction_tensor)
    zeros = array_ops.zeros_like(sigmoid_p, dtype=sigmoid_p.dtype)
    
    # For poitive prediction, only need consider front part loss, back part is 0;
    # target_tensor > zeros <=> z=1, so poitive coefficient = z - p.
    pos_p_sub = array_ops.where(target_tensor > zeros, target_tensor - sigmoid_p, zeros)
    
    # For negative prediction, only need consider back part loss, front part is 0;
    # target_tensor > zeros <=> z=1, so negative coefficient = 0.
    neg_p_sub = array_ops.where(target_tensor > zeros, zeros, sigmoid_p)
    per_entry_cross_ent = - alpha * (pos_p_sub ** gamma) * tf.math.log(tf.clip_by_value(sigmoid_p, 1e-8, 1.0)) \
                          - (1 - alpha) * (neg_p_sub ** gamma) * tf.math.log(tf.clip_by_value(1.0 - sigmoid_p, 1e-8, 1.0))
    return tf.reduce_sum(per_entry_cross_ent)

In [ ]:
model = Sequential()

model.add(Conv2D(32, (3, 3), input_shape=(200, 200,1)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))
model.add(Conv2D(32,(3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(5,5)))
# model.add(Dropout(0.25))

model.add(Flatten())


# model.add(Dense(100))
# model.add(Activation('relu'))

model.add(Dense(50))
model.add(Activation('relu'))

model.add(Dense(6))
model.add(Activation('sigmoid'))

# model.compile(loss='categorical_crossentropy',
#               optimizer='adam',
#               metrics=['accuracy'])
model.compile(loss=focal_loss,
               optimizer='adam',
               metrics=['accuracy'])

#model.fit(X,y_train,batch_size=32,epochs=6,validation_split=0.5)
model.summary()

## Layer maps from training

In [ ]:
def plot_filters_keras(model):
    for layer in model.layers:
        if 'conv' in layer.name:
            weights, bias= layer.get_weights()
            print(layer.name, weights.shape)
        
            #normalize filter values between  0 and 1 for visualization
            f_min, f_max = weights.min(), weights.max()
            filters = (weights - f_min) / (f_max - f_min)  
            print(filters.shape[3])
            filter_cnt=1
        
            #plotting all the filters
            for i in range(filters.shape[3]):
                #get the filters
                filt=filters[:,:,:, i]
                #plotting each of the channel, color image RGB channels
                for j in range(filters.shape[2]):
                    ax= plt.subplot(filters.shape[3], filters.shape[2], filter_cnt)
                    ax.set_xticks([])
                    ax.set_yticks([])
                    plt.imshow(filt[:,:, j])
                    filter_cnt+=1
        plt.show()

In [ ]:
plot_filters_keras(model)

In [ ]:
def plot_grid_keras(datagenerator, image_index=0):
    data = datagenerator[image_index][0]
    successive_outputs = [layer.output for layer in model.layers[1:]]

    visualization_model = tf.keras.models.Model(inputs = model.input, outputs=successive_outputs)
    successive_feature_maps = visualization_model.predict(data)
    layer_names = [layer.name for layer in model.layers]
    for layer_name, feature_map in zip(layer_names, successive_feature_maps):
        print(feature_map.shape)
        if(len(feature_map.shape) == 4):
            n_features = feature_map.shape[-1]
            size = feature_map.shape[1]
            display_grid = np.zeros((size, size * n_features))
            for i in range(n_features):
                data = feature_map[0, :, :, i]
                data -= data.mean()
                data /= data.std()
                data *= 64
                data += 128
                data = np.clip(data, 0, 255).astype('uint8')
                display_grid[:, i * size : (i + 1) * size] = data
            scale = 20. / n_features
            plt.figure(figsize=(scale*n_features, scale))
            plt.title(layer_name)
            plt.grid(False)
            plt.imshow(display_grid, aspect='auto', cmap='bone')

In [ ]:
plot_grid_keras(datagenerator=traingen)

In [ ]:
history=model.fit_generator(generator=traingen,validation_data=validgen,use_multiprocessing=False,
                    workers=-1,epochs=KERAS_EPOCHS)
